In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from keras.layers import Dense, Input, Flatten, Conv2D,Conv1D, MaxPooling2D,MaxPooling1D,GlobalMaxPooling1D
from keras.layers import Reshape, Dropout, Concatenate, LSTM,Bidirectional,BatchNormalization
from keras.layers import Flatten,Activation
from keras.models import Model
from keras.models import Sequential
from keras import regularizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
import os
import json
from IPython.core.display import display, HTML
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


/kaggle/input/emotionrecognition/dev.json
/kaggle/input/emotionrecognition/train.json


In [2]:
dir = '/kaggle/input/emotionrecognition/'
PAD_ROWS=1800 
PAD_COLS=26

In [3]:
f = open(os.path.join(dir, 'train.json'), encoding="utf8")
data = json.load(f)

In [4]:
df = pd.DataFrame(data)
df= df.T
TOTAL_DATA=df.shape[0]
df.head()


,valence,activation,features
0,0,1,"[[5.502810676891276, 5.389630715979907, 5.8907..."
1,1,1,"[[5.059076172970736, 5.288492317702101, 4.2633..."
2,0,1,"[[4.218546271669202, 4.961436495859291, 3.6650..."
3,1,0,"[[4.650364321573866, 4.523905028353254, 5.0168..."
4,0,1,"[[3.900221957277269, 2.7325726489808124, 2.565..."


In [5]:
TOTAL_DATA

7800

In [6]:
feat = np.zeros((TOTAL_DATA,PAD_ROWS,PAD_COLS),dtype=np.float32)

feature_set=pd.DataFrame(np.zeros((TOTAL_DATA,2),dtype=np.float32),columns=['features','label'])
feature_set['features']=df['features']
code=0
for i in tqdm_notebook(range(TOTAL_DATA)):
    
    [r,c]=np.array(df['features'][i]).shape
    feat[i,:r,:c]=np.array(normalize(df['features'][i]))
    feature_set['features'][i]=feat[i]
    v=df['valence'][i]
    a=df['activation'][i]
    
    if v==0 and a==0:
        code=0
    elif v==0 and a==1:
        code=1
    elif v==1 and a==0:
        code=2
    elif v==1 and a==1:
        code=3
    feature_set['label'][i]=code

In [7]:
print(feat.shape)

(7800, 1800, 26)


<h4> Create Separate Labelset for **Valence** and **Activation** </h4>

In [8]:
labels=feature_set['label'].astype(int)
valence=np.array(df['valence'])
activation=np.array(df['activation'])
print(valence)
print(activation)


[0 1 0 ... 0 0 1]
[1 1 1 ... 1 1 1]


In [9]:
X = feat


In [10]:
X_train, X_test, val_train, val_test = train_test_split(X, valence, test_size=0.10, random_state=96)
X_train, X_test, act_train, act_test = train_test_split(X, activation, test_size=0.10, random_state=96)
input_dim=X_train[0].shape

**Creating the Model**

In [11]:
# model = Sequential()

# model.add(Bidirectional(LSTM(256, input_shape=input_dim,  dropout=0.5, recurrent_dropout=0.2,return_sequences=True)))

# # model.add(LSTM(64, dropout=0.5, recurrent_dropout=0.2,return_sequences=True))

# model.add(keras.layers.SimpleRNN(32, dropout=0.5, recurrent_dropout=0.2, return_sequences=False))


# model.add(Dense(32,activation=tf.nn.relu))
# model.add(Dropout(0.5))

# # model.add(Dense(16,activation=tf.nn.relu))
# # model.add(Dropout(0.5))

# # # model.add(Dense(8,activation=tf.nn.relu))
# # # model.add(Dropout(0.5))

# model.add(Dense(4, activation=tf.nn.softmax))
# model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])


In [12]:
# model = Sequential()
# model.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=input_dim))
# model.add(Dropout(0.2))
# model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# model.add(Dropout(0.2))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(filters=32, kernel_size=6, activation='relu'))
# model.add(Dropout(0.5))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(filters=8, kernel_size=6, activation='relu'))
# model.add(Dropout(0.5))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Flatten())

# model.add(Dense(16, activation='relu'))
# model.add(Dropout(0.8))
# model.add(Dense(4, activation='softmax'))
# model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [19]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=6, activation='relu', input_shape=input_dim))
model.add(BatchNormalization())
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv1D(filters=16, kernel_size=2, activation='relu'))
model.add(BatchNormalization())
model.add(Conv1D(filters=16, kernel_size=2, activation='relu'))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='softmax'))
# model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

ValueError: Input 0 is incompatible with layer lstm_1: expected ndim=3, found ndim=2

**Train the Model**


In [14]:
## ------ Network 1 ------##
## https://github.com/CVxTz/audio_classification/blob/master/code/keras_cnn_starter.py
# model = Sequential()
# model.add(Conv1D(filters=16, kernel_size=9, activation='relu', padding="valid",input_shape=input_dim))
# model.add(Conv1D(filters=16, kernel_size=9,padding="valid", activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling1D(pool_size=16))
# #model.add(Dropout(0.1))

# model.add(Conv1D(filters=32, kernel_size=3,padding="valid", activation='relu'))
# model.add(Conv1D(filters=32, kernel_size=3,padding="valid", activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling1D(pool_size=4))
# #model.add(Dropout(0.1))

# model.add(Conv1D(filters=32, kernel_size=3,padding="valid", activation='relu'))
# model.add(Conv1D(filters=32, kernel_size=3,padding="valid", activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling1D(pool_size=4))
# #model.add(Dropout(0.1))

# model.add(Conv1D(filters=256, kernel_size=3,padding="valid", activation='relu'))
# model.add(Conv1D(filters=256, kernel_size=3, padding="valid",activation='relu'))
# model.add(BatchNormalization())
# model.add(GlobalMaxPooling1D())
# #model.add(Dropout(0.2))

# model.add(Dense(512, activation='relu'))
# model.add(BatchNormalization())

# model.add(Dense(128, activation='relu'))
# model.add(BatchNormalization())

# model.add(Dense(1, activation='softmax'))

In [15]:
## ------ Network 2 ------##
## CNN-RNN Model ##
## https://github.com/priya-dwivedi/Music_Genre_Classification/blob/master/CRNN_model.ipynb

# model = Sequential()
# model.add(Conv1D(filters=64, kernel_size=6, activation='relu', kernel_regularizer=regularizers.l2(0.001),input_shape=input_dim))
# model.add(BatchNormalization(momentum=0.9))
# model.add(Activation('relu'))
# model.add(MaxPooling1D(2))
# model.add(Dropout(0.4))
# model.add(LSTM(96, return_sequences=False))
# model.add(Dropout(0.4))
# model.add(Dense(64, kernel_regularizer=regularizers.l2(0.001), name='dense1'))
# model.add(Dropout(0.4))
# model.add(Dense(1, activation='softmax'))



In [16]:
opt = keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

In [18]:
model.fit(X_train, val_train, epochs=100,verbose=1,validation_data=(X_test, val_test))  # train the model

Train on 7020 samples, validate on 780 samples
Epoch 1/100
7020/7020 [==============================] - 8s 1ms/step - loss: 8.7915 - accuracy: 0.4266 - val_loss: 8.0401 - val_accuracy: 0.4756
Epoch 2/100
7020/7020 [==============================] - 3s 476us/step - loss: 8.7915 - accuracy: 0.4266 - val_loss: 8.0401 - val_accuracy: 0.4756
Epoch 3/100
7020/7020 [==============================] - 3s 482us/step - loss: 8.7915 - accuracy: 0.4266 - val_loss: 8.0401 - val_accuracy: 0.4756
Epoch 4/100
7020/7020 [==============================] - 3s 476us/step - loss: 8.7915 - accuracy: 0.4266 - val_loss: 8.0401 - val_accuracy: 0.4756
Epoch 5/100
7020/7020 [==============================] - 3s 478us/step - loss: 8.7915 - accuracy: 0.4266 - val_loss: 8.0401 - val_accuracy: 0.4756
Epoch 6/100
7020/7020 [==============================] - 3s 482us/step - loss: 8.7915 - accuracy: 0.4266 - val_loss: 8.0401 - val_accuracy: 0.4756
Epoch 7/100
7020/7020 [==============================] - 3s 480us/step - 

KeyboardInterrupt: 

In [ ]:
model.summary()

In [ ]:
val_loss, val_acc = model.evaluate(X_test, y_test)  # evaluate the out of sample data with model
print("Loss:",val_loss)  # model's loss (error)
print("Accuracy:",val_acc*100)  # model's accuracy

In [ ]:
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=6, activation='relu', input_shape=input_dim))
model.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(X, Y, validation_split=0.33, epochs=150, batch_size=10, verbose=0)
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()